In [12]:
# import libraries related to querying links and downloading files from the web
import pandas as pd
from pipmag import selector_utils as su
from pipmag import ads_utils as ads
from pipmag import file_utils as fu
# If you get the error ModuleNotFoundError: No module named 'pipmag', run the following line and restart the kernel:
# %pip install -e ..

In [13]:
# get the latest csv file generated by running from the root folder
# python python -m pipmag.gen_la_palma_df
la_palma_obs_data_file = '../data/la_palma_obs_data.csv'
df = fu.read_and_format_csv(la_palma_obs_data_file)
df.info()

In [16]:
# create a widget to display movies based on year, month, day and time
# and to update the target, instrumnets and comments columns of the dataframe
selector = su.VideoSelector2(df, ['target', 'instruments', 'polarimetry', 'comments'])
selector.create_widget()

Dropdown(description='Year:', options=(2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023), valu…

Dropdown(description='Month:', options=(), value=None)

Dropdown(description='Day:', options=(), value=None)

Dropdown(description='Time:', options=(), value=None)

Dropdown(description='Links:', options=(), value=None)

...

Text(value='', description='target:')

Text(value='', description='instruments:')

Text(value='', description='polarimetry:')

Text(value='', description='comments:')

Button(description='Update', style=ButtonStyle())

In [ ]:
# 🔍 ADS Search
index = 34
search = ads.ADS_Search(df)
search.get_results(index, pretty_print=True)

In [ ]:
## Rewrite keywords in target column to general terms 
def rewrite_keywords(text, target_keywords, relpacewith):
    for keyword in target_keywords:
        if keyword in text:
            text = text.replace(keyword, relpacewith)
    return text

# Target keywords: 
ACTIVE_REGION_KEYWORDS = {'active region', 'Active region', 'AR'}
QUIET_SUN_KEYWORDS     = {'quiet Sun', 'quiet sun', 'QS', 'Quiet sun'}
SUNSPOT_KEYWORDS       = {'sunspot', 'Sunspot', 'SS', 'ss', 'SUnspot'}
# Replace keywords in target column with more general terms
df['target'] = df['target'].apply(lambda x: None if pd.isna(x) else rewrite_keywords(x, ACTIVE_REGION_KEYWORDS, "Active Region"))
df['target'] = df['target'].apply(lambda x: None if pd.isna(x) else rewrite_keywords(x, QUIET_SUN_KEYWORDS, "Quiet Sun"))
df['target'] = df['target'].apply(lambda x: None if pd.isna(x) else rewrite_keywords(x, SUNSPOT_KEYWORDS, "Sunspot"))

In [ ]:
# make a copy of the dataframe
df_copy = df.copy()

# List of columns to convert from lists to strings
columns_to_convert = ['links', 'video_links', 'image_links', 'instruments']
for col in columns_to_convert:
    df_copy[col] = df_copy[col].apply(lambda x: ';'.join(x))

# save the updated dataframe as a .csv file
df_copy.to_csv(la_palma_obs_data_file, index=False)